In [47]:
# remove all current wavs

import os

amountOfChunks = len([entry for entry in os.listdir("wavs") if os.path.isfile(os.path.join("wavs", entry))])
print('amount of files: ', amountOfChunks)

for i in range(amountOfChunks):
    os.remove(f"wavs/chunk{i}.wav")

amount of files:  70


In [34]:
# test
import subprocess as command

command.run("sox-14-4-2/sox fn00005.wav chunk0.wav remix 1 trim 29.3000 1.6326999999999998")

In [40]:
# Tijdelijk de class hierin geplakt, het importen werkt nog niet helemaal

import pandas as pd
import numpy as np
import string
import wave
import pydub
import subprocess as command

class WER:
    """
    This class will convert an ORT file to a Pandas DataFrame which consist of spoken words, and the begin and endtime in which they were spoken.
    """
    #filters = ["!", "*", "xxx",]
    
    def ort2df(self, file_path: str) -> pd.DataFrame:
        file = open(file_path, 'r',encoding='latin-1')
        lines = [line.replace("\"", "").strip() for line in file.readlines()] # Remove the quatation marks and newline character
        records = []
        interval_count = 0
        print("-----------------------------")
        print(file_path)
        print("-----------------------------")
        
        i = 0
        for line in lines: # Check the lines untill the second IntervalTier appears
            if line != "IntervalTier":
                i += 1
            elif interval_count == 1:
                break
            else:
                interval_count = 1
        
        i += 6 # Skip some lines of data which arent used
        j = 0 # Unique id
        
        while True:
            if lines[i] == "IntervalTier" or lines[i] == "BACKGROUND" or lines[i] == "TextTier" or lines[i] == "COMMENTS":
                df = pd.DataFrame(np.array(records))
                df.columns = ["Id", "Text", "StartTime", "EndTime"]
                return df
            
            text = lines[i+2]
            #if not (any(x in text for x in self.filters) or text == ""): # Checks if the text contains a filter character, or is an empty text
            if (text != ""): # Checks if the text is empty
                text = text.translate(str.maketrans('', '', string.punctuation))
                record = [j, text.lower(), lines[i], lines[i+1]]
                #print(record)
                j += 1
                records.append(record)
            i += 3
            
            
    def wav_splitter(self, file_path: str, df: pd.DataFrame):
        for index, row in df.iterrows():
            cmd = "sox-14-4-2/sox {} wavs/chunk{}.wav remix 1 trim {} {}".format(file_path, index, row["StartTime"], float(row["EndTime"]) - float(row["StartTime"]))
            print(cmd)
            command.run(cmd)

    def word_count(self, words):
        word_count = words['Text'].iloc[:-1]
        df = pd.value_counts((word_count))
        print(df)


In [49]:

from torchmetrics import WordErrorRate
import json
file = 'fn000061'
wer = WER()
# For on the server:
#ort = '../datasets/jasmin/Data/data/annot/text/ort/comp-p/nl/fn000050.ort'
# wav = '../datasets/jasmin/Data/data/audio/wav/comp-p/nl/fn000050.wav'

# splitter:
wer.wav_splitter(file + ".wav", wer.ort2df(file + ".ort"))
# ort to dataframe:
df = pd.DataFrame(wer.ort2df(file+".ort"))
print(df)

# deletes just used ort and wav
# os.remove(f"{file}.wav")
# os.remove(f"{file}.ort")

# # function to check results
# def checker(id):
#     with open(f"wavresults/testPredictionF{id}.json", 'r', encoding='utf-8') as f:
#         jsn = json.loads(f.read())
#     return(df['Text'][id], jsn['text'])

# # all results together in 1 long string for total WER
# truth = ''
# prediction = ''

# results_df = pd.DataFrame(columns = ['WER', 'Truth', 'Predicted'])


# for i in range (len(df.index)):
#     expected = df['Text'][i]
#     truth = truth + ' ' + expected
#     with open(f"wavresults/testPredictionF{i}.json", 'r', encoding='utf-8') as f:
#         jsn = json.loads(f.read())
        
#     target = jsn['text']
#     prediction = prediction + ' ' + target
    
    
#     metric = WordErrorRate()
#     print(metric(expected, target).item(), df['Text'][i], jsn['text'])
#     results_df.loc[i] = metric(expected, target).item(), df['Text'][i], jsn['text']


# print('total word error rate: ', metric(truth, prediction).item())
# results_df.to_csv("wer.csv", index=False)

-----------------------------
fn000059.ort
-----------------------------
sox-14-4-2/sox fn000059.wav wavs/chunk0.wav remix 1 trim 0.7533 0.9693999999999999
sox-14-4-2/sox fn000059.wav wavs/chunk1.wav remix 1 trim 26.3429 1.468399999999999
sox-14-4-2/sox fn000059.wav wavs/chunk2.wav remix 1 trim 46.7542 1.6248000000000005
sox-14-4-2/sox fn000059.wav wavs/chunk3.wav remix 1 trim 64.0464 1.28779999999999
sox-14-4-2/sox fn000059.wav wavs/chunk4.wav remix 1 trim 65.6399 1.0563999999999965
sox-14-4-2/sox fn000059.wav wavs/chunk5.wav remix 1 trim 73.6433 1.1433999999999997
sox-14-4-2/sox fn000059.wav wavs/chunk6.wav remix 1 trim 75.7782 2.250699999999995
sox-14-4-2/sox fn000059.wav wavs/chunk7.wav remix 1 trim 79.0278 1.1820000000000022
sox-14-4-2/sox fn000059.wav wavs/chunk8.wav remix 1 trim 81.6395 0.8425000000000011
sox-14-4-2/sox fn000059.wav wavs/chunk9.wav remix 1 trim 83.1727 1.5524999999999949
sox-14-4-2/sox fn000059.wav wavs/chunk10.wav remix 1 trim 85.5557 1.4682999999999993
sox-14-

In [37]:
# awd.word_count(a)


In [38]:
# import pandas as pd
# import numpy as np
# import string

# data = pd.read_json('additionalInfo.json')

# class WordCheck:
#     def convert(self, filepath:str):
#         data = pd.read_json(filepath)
#         df_new = data[['word', 'start_time', 'end_time']]
#         print("-----------------------------")
#         print(filepath)
#         print("-----------------------------")
        
        
#         list1 = [list(i) for i in zip(df_new['word'], df_new['start_time'], df_new['end_time'])]
#         df = pd.DataFrame(np.array(list1))
#         df.columns = ["Text", "StartTime", "EndTime"]
        
#         j = 0
#         for x, y,q in zip(df['Text'], df['StartTime'], df['EndTime']):
#             records = [j,x,y,q]
#             print(records)
#             j+=1
#         return df
            
            
# file = 'additionalInfo.json'
# WordChecker = WordCheck()
# w = WordChecker.convert(file)

In [39]:
# print(w)